<a href="https://colab.research.google.com/github/andrewcgaitskell/celldb/blob/master/CellMaker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Libraries

In [52]:
!pip install xlsxwriter
!pip install xlrd
!pip install matplotlib

In [0]:
import pandas as pd
import numpy as np

In [0]:
import xlsxwriter
import xlrd
import matplotlib

# Introduction

In [0]:
from datetime import datetime
import os

In [56]:
from datetime import datetime
my_date = datetime.now()
print(my_date.isoformat())

2020-06-15T11:44:25.694340


In [57]:
print(my_date.strftime('%Y-%m-%dT%H:%M:%S.%f%z'))

2020-06-15T11:44:25.694340


In [58]:
runid = "R" + my_date.strftime('%Y%m%d%H%M%S%f%z')
runid

'R20200615114425694340'

In [59]:
dirname = os.getcwd()
print(dirname)

/content


## Upload File

# Excel Data Types

The following functions are use to work out the data type of an excel cell.

In [0]:
def validate_date(date_text):
    try:
        format_string = '%Y-%m-%d %H:%M:%S'
        datetime.strptime(date_text, format_string)
        return "OK"
    except ValueError:
        return None

def GetDataType(row_in):
    valueasstring = str(row_in['CellValue'])
    if validate_date(valueasstring) != None:
        return 'Date'
    elif valueasstring.isnumeric():
        return 'Number'
    else:
        return 'Text'

def add1_2row(row_in):
    return row_in['RowID'] + 1


def add1_2column(row_in):
    return row_in['ColumnID'] + 1

In [63]:
from ipywidgets import FileUpload
upload = FileUpload()
upload

FileUpload(value={}, description='Upload')

In [0]:
import io
filedf = pd.read_excel(upload.data[0],sheet_name=None, header=0, encoding='utf-8', errors='ignore')


In [66]:
filedf

{'AgeGenderSheet':    Unnamed: 0           Unnamed: 1 Unnamed: 2    Unnamed: 3         Unnamed: 4
 0         NaN                  NaN        NaN           NaN                NaN
 1         NaN                  NaN        NaN           NaN                NaN
 2         NaN                  NaN        NaN           NaN                NaN
 3         NaN            Age Table        NaN           NaN                NaN
 4         NaN                  NaN       Name           Age                NaN
 5         NaN                  NaN       Paul           -52                NaN
 6         NaN                  NaN      Ringo            22                NaN
 7         NaN                  NaN      Brian            89                NaN
 8         NaN                  NaN        NaN           NaN                NaN
 9         NaN                  NaN        NaN           NaN                NaN
 10        NaN                  NaN        NaN           NaN                NaN
 11        NaN        

# Read All Sheets into Data Frame

In [0]:
sheetnames = list(filedf.keys())

In [68]:
print(sheetnames)

['AgeGenderSheet', 'HomeSheet', 'JobSheet', 'MaritalDepartmentSheet', 'DataMap', 'TemplateValidationMap', 'DataSet', 'Rules', 'Palette', 'PickList']


Each Sheet is retrieved into Dataframes

DataMapDF = specdf['DataMap']
TemplateValidationMapDF = specdf['TemplateValidationMap']
DataSetDF = specdf['DataSet']
RulesDF = specdf['Rules']
PaletteDF = specdf['Palette']
PickListDF = specdf['PickList']
datadf = pd.DataFrame()


The following converts the whole spreadsheet into a cell database and this is used throughout for all data frame population.

In [72]:
celldbstagedf = pd.DataFrame()
sheetdf = filedf['DataMap'].copy()
sheetdf["RowID"] = sheetdf.index
sheetdf

,SheetName,ReferenceText,RowFromReferenceText,ColumnFromReferenceText,TemplateText,DataGroup,DataCategory,DataScope,Column Header,Dataset,RowID
0,AgeGenderSheet,Age Table,2,1,Name,Ages,NaN,NaN,Name,Ages,0
1,AgeGenderSheet,Age Table,2,2,Age,Ages,NaN,NaN,Age,Ages,1
2,AgeGenderSheet,Gender Table,2,0,Name,Genders,NaN,NaN,Name,Genders,2
3,AgeGenderSheet,Gender Table,2,1,Gender,Genders,NaN,NaN,Gender,Genders,3
4,AgeGenderSheet,DOB Table,2,0,Name,DoB,NaN,NaN,Name,DoB,4
5,AgeGenderSheet,DOB Table,2,1,D.O.B,DoB,NaN,NaN,D.O.B,DoB,5
6,MaritalDepartmentSheet,Marital Status Table,2,0,Name,Marital Status,NaN,NaN,Name,MaritalStatus,6
7,MaritalDepartmentSheet,Marital Status Table,2,1,Status,Marital Status,NaN,NaN,Status,MaritalStatus,7
8,MaritalDepartmentSheet,Department Table,2,0,Name,Department,NaN,NaN,Name,Department,8
9,MaritalDepartmentSheet,Department Table,2,1,Department,Department,NaN,NaN,Department,Department,9


In [75]:
for currentsheetname in sheetnames:

    sheetdf = filedf[currentsheetname].copy()
    sheetdf["RowID"] = sheetdf.index
    celldb = pd.melt(sheetdf, id_vars=['RowID'])
    celldb = celldb.rename(columns={"variable": "ColumnID", "value": "CellValue"})
    celldb["CellType"] = "Unset"
    celldb["FileName"] = runid
    celldb["SheetName"] = currentsheetname

    #try:
    celldb["CellType"] = celldb.apply(GetDataType, axis=1)
    celldb["RowID"] = celldb.apply(add1_2row, axis=1)
    celldb["ColumnID"] = celldb.apply(add1_2column, axis=1)

    #except:

    #    celldb["CellType"] = 'unknown'
    #    celldb["RowID"] = 0
    #    celldb["ColumnID"] = 0

    celldbstagedf = celldbstagedf.append(celldb, ignore_index=True)

    celldb = pd.DataFrame()



TypeError: ignored

In [74]:
celldbstagedf

,RowID,ColumnID,CellValue,CellType,FileName,SheetName
0,0,0,NaN,unknown,R20200615114425694340,AgeGenderSheet
1,0,0,NaN,unknown,R20200615114425694340,AgeGenderSheet
2,0,0,NaN,unknown,R20200615114425694340,AgeGenderSheet
3,0,0,NaN,unknown,R20200615114425694340,AgeGenderSheet
4,0,0,NaN,unknown,R20200615114425694340,AgeGenderSheet
...,...,...,...,...,...,...
1050,0,0,Married,unknown,R20200615114425694340,PickList
1051,0,0,London,unknown,R20200615114425694340,PickList
1052,0,0,Liverpool,unknown,R20200615114425694340,PickList
1053,0,0,Male,unknown,R20200615114425694340,PickList


# Get Picklist Values

Picklist are like a mini Master Data Management System

They define the acceptable values for a partcular Field in Excel

The challenge here was to check one column to a another column that contained a list.  I go there in the end.

In [0]:
PickListCells = pd.DataFrame()
# This process uses the full cell database as the column headings are the Picklist Names
PickListCells = fullcelldb[(fullcelldb['SheetName'] == 'PickList')].copy()
# PickListCellsList = PickListCells.groupby('ColumnID')['CellValue'].apply(lambda group_series: group_series.tolist()).reset_index()

PickListCellsList = PickListCells.groupby('ColumnID').agg({'CellValue':lambda x: list(x)})

# these were odd, but glad I work them out, it is important to realise that these are operations on a number of rows, not just one
# maybe it was the axis that made the difference!
PickListCellsList['KeyWord'] = PickListCellsList.apply(lambda row : row['CellValue'][0], axis = 1)
PickListCellsList['FormulaKeyWord'] = PickListCellsList.apply(lambda row : 'PickList', axis = 1)
PickListCellsList['List'] = PickListCellsList.apply(lambda row : row['CellValue'][1:], axis = 1)
# PickListCellsList['KeyWord'] = PickListCellsList['CellValue']
#PickListCellsList= pd.DataFrame(PickListCellsList, columns=['KeyWord', 'List'])
PickListCellsList = pd.DataFrame(PickListCellsList, columns=['KeyWord', 'FormulaKeyWord', 'List'])
PickListCellsList.head()

In [0]:
# These were attempts at creating a suitable list for the function to check to

# PickListCellsList['Result'] = PickListCellsList['KeyWord'].isin(PickListCellsList['List']).map({True: 0, False: 1})

# PickListCellsList['Result'] = PickListCellsList['KeyWord'].str.contains('|'.join(PickListCellsList['List']))
# PickListCellsList['Result'] = PickListCellsList['KeyWord'].str.contains('|'.join(x) for x in PickListCellsList['List'])
# ["".join(x) for x in lst]
# PickListCellsList['ListAsString'] = PickListCellsList['List'].str.join('-')
# PickListCellsList['ListAsTuple'] = tuple(PickListCellsList['CellValue'])


In [0]:
#MyList = pd.DataFrame()
#MyList =  PickListCellsList
# still do not know why the following DOES NOT WORK!
# MyList['Result'] = PickListCellsList['ListAsString'].isin(PickListCellsList['KeyWord'])

# leaving this as a rememberance of the 6 hours spent on this!
# MyList['Result1'] = PickListCellsList['KeyWord'].apply(lambda x: any(i in x for i in PickListCellsList['List']))
# MyList['Result2'] = PickListCellsList['ListAsString'].str.contains(PickListCellsList['KeyWord'])
# mask = df['A'].apply(lambda x: any(i in x for i in search_list))
# PickListCellsList['Result2'] = PickListCellsList['ListAsTuple'].str.contains(PickListCellsList['KeyWord'])
# PickListCellsList['Result2'] = PickListCellsList['KeyWord'].str.contains(tuple(PickListCellsList['List']))
# PickListCellsList['ListAsTuple'].isin(PickListCellsList['KeyWord'])
# MyList['Result2'] = PickListCellsList['List'].apply(lambda x: True if (PickListCellsList.KeyWord in x).any() else False)
######### MyList['sing'] = 'Single'
# MyList['res1'] = MyList.apply(lambda ts: any(ts == 'Single'), axis=1)
# MyList['res2'] = MyList.apply(lambda ts: MyList['KeyWord'].str.contains('Single'), axis=1)
######### MyList['res3'] = MyList.apply(lambda x: x['sing'] in x['ListAsTuple'], axis=1)  ###############
######### MyList['res4'] = MyList.apply(lambda x: x['sing'] in x['List'], axis=1).map({True: 1, False: 0}) ###############
# # # # dataset.apply(lambda x: x['CellValue'] in x['PickList'], axis=1).map({True: 1, False: 0})
# data['Activity].str.contains('skin diving')
# lambda x : True if (x > 10 and x < 20) else False
#MyList.head()
# https://stackoverflow.com/questions/48460234/pandas-apply-valueerror-the-truth-value-of-a-series-is-ambigous

# Get Data Map

In [0]:
DataMapCells = pd.DataFrame()
DataMapCells = fullcelldb[(fullcelldb['SheetName'] == 'DataMap')].copy()
DataMapCells.head()

Below extracts a Sheet from the cell database and reshapes it to it original tabular form.

In [0]:
DataMap = pd.DataFrame()
DataMap = DataMapCells.pivot_table(index = ['RowID'], values = 'CellValue',columns = 'ColumnID', aggfunc = 'max')
ColumnHeadings = DataMap.iloc[0]
ColumnHeadingsList = list(ColumnHeadings)
print(ColumnHeadingsList)
ColumnHeadingsListTuple = enumerate(ColumnHeadingsList,start=1)
ColumnHeadingsListDict = dict(ColumnHeadingsListTuple)
print(ColumnHeadingsListDict)
DataMap = DataMap.rename(columns=ColumnHeadingsListDict)
DataMap = DataMap.drop([1])
DataMap.head()

# Make Rules Database

In [0]:
RulesDF.head()

The above is the Rules definition sheet. It used KeyWords to define the checks.  Below forms a list of KeyWords that are relevant to a particular field in a DataSet

In [0]:
rulesdb = pd.DataFrame()
rulesdb = pd.melt(RulesDF, id_vars=['RuleID', 'DataSet', 'FieldID'])
rulesdb = rulesdb.rename(columns={"variable": "RuleType", "value": "KeyWord"})
rulesdb = rulesdb[rulesdb.KeyWord.notnull()]
# df['A'] = df.apply(lambda x: x['B'] if x['A']==0 else x['A'], axis=1)
# The following adds the KeyWord Null into the Rules list - if Null is N, this means Nulls are Not Allowed
rulesdb['KeyWord'] = rulesdb.apply(lambda x: x['RuleType'] if x['KeyWord']=='N' else x['KeyWord'], axis=1)

PickListCellsList

rulesdb = pd.merge(rulesdb,PickListCellsList, on=['KeyWord'],how='left')

rulesdb.head(5)

# Add Code and Formulae to Rules Database

Palette defines the Rule Text and Rule Formula.  We have implemented rules on Alteryx, SQL, Python & Javascript

In [0]:
palette = pd.DataFrame()
palette = pd.merge(DataSetDF, rulesdb, on=['FieldID', 'DataSet'])
palette['KeyWord'] = rulesdb.apply(lambda x: x['FormulaKeyWord'] if x['FormulaKeyWord']=='PickList' else x['KeyWord'], axis=1)
palette.head(5)

In [0]:

palette = pd.merge(palette, PaletteDF, on=['KeyWord'],how='left')


In [0]:
palette.head(5)

# Get Start Anchor Cells

Start Anchors are where tables are mapped from, every dataset in the spreadsheet must be anchored to a uniquely identifiable cell in a particular sheet.

In [0]:
startanchors = pd.DataFrame()
startanchors = pd.DataFrame(DataSetDF, columns=['DataSet', 'SheetName', 'ReferenceTextStart'])
startanchors = startanchors.rename(columns={"ReferenceTextStart": "CellValue"})
startanchors = startanchors.drop_duplicates()
startanchors.head()


# Get All End Anchor Cells

End anchors are where a dataset finishes. Probably at the next Start Anchor.
This does mean that blank rows are validated and one needs to filter these out later. 

In [0]:
endanchors = pd.DataFrame()
endanchors = pd.DataFrame(DataSetDF, columns=['DataSet', 'SheetName', 'ReferenceTextEnd'])
endanchors = endanchors.rename(columns={"ReferenceTextEnd": "CellValue"})
endanchors = endanchors.drop_duplicates()
endanchors.head()

# Find Start Anchors

Now we look for the start and finish anchor cells for each dataset.

In [0]:
findstartanchors = pd.DataFrame()
findstartanchors = pd.merge(startanchors, fullcelldfstage, on=['SheetName', 'CellValue'])
findstartanchors = findstartanchors.rename(columns={"RowID": "AnchorRow", "ColumnID": "AnchorColumn"})
findstartanchors = pd.DataFrame(findstartanchors, columns=['DataSet', 'SheetName', 'AnchorRow', 'AnchorColumn'])
findstartanchors.head()

# Find End Anchors

Some datasets have no finish anchors as they are the last table in the sheet.

In [0]:
findendanchors = pd.DataFrame()
findendanchors = pd.merge(endanchors, fullcelldfstage, on=['SheetName', 'CellValue'])
findendanchors = findendanchors.rename(columns={"RowID": "EndRow"})
findendanchors = findendanchors[findendanchors.CellValue.notnull()]
findendanchors = pd.DataFrame(findendanchors, columns=['DataSet', 'SheetName', 'EndRow'])
findendanchors.head()


# Join Anchors Together

Join the anchors to allow them to be joined with the data table
As stated above, some DataSets have NaN as the EndRow as there is no further data in the sheet

In [0]:
anchors = pd.DataFrame()
anchors = pd.merge(findstartanchors, findendanchors, on=['DataSet', 'SheetName'], how='left')
anchors.head()

# Using Anchors and Dataset Create Map to Find Data

The DataSet defines where the DataSets can be found from the Start Anchor Cells
The following joins the found cells with the DataSet map.

In [0]:
datasetmap = pd.DataFrame()
datasetmap = pd.merge(DataSetDF, anchors, on=['DataSet', 'SheetName'], how='left')
datasetmap = pd.DataFrame(datasetmap,
                          columns=['DataSet', 'SheetName', 'FieldID',
                                   'AnchorRow', 'AnchorColumn','EndRow',
                                   'RowFromReferenceText',
                                   'ColumnFromReferenceText','FieldPosition'])


## Show Dataset Map

In [0]:
datasetmap.head()

# Find where Data Starts and what Columns Fields are In

One needs to now calculate the actual rows and columns of where the DataSet is.
The above table is relative to the Anchor Cells

In [0]:
datasetmap['StartRow'] = datasetmap['AnchorRow'] + datasetmap['RowFromReferenceText']
datasetmap['FieldColumn'] = datasetmap['AnchorColumn'] + datasetmap['ColumnFromReferenceText']
dataset = pd.DataFrame()
dataset = pd.merge(fullcelldfstage, datasetmap, on=['SheetName'], how='left')
## the above may seem a cartesian join and may need to be looked at if this slows the process down

# Filter Out Data after End Anchor Row

In [0]:
dataset = dataset[(dataset.RowID >= dataset.StartRow) &
(dataset.ColumnID == dataset.FieldColumn) &
((dataset.RowID < dataset.EndRow) | pd.isnull(dataset.EndRow))]
# the | above is an OR as if there is no EndRow for the dataset you want to
# allow the data through

In [0]:
palette.head(5)

# Create a CellValue as Number for use in Formulae

In [0]:
dataset = pd.DataFrame(dataset,columns=['DataSet', 'SheetName', 'FieldID', 'RowID', 'ColumnID','CellValue', 'CellType'])
dataset['CellValueAsNumber'] = pd.to_numeric(dataset['CellValue'], errors='coerce')
dataset.head(5)

# Create a List of Checks to do

In [0]:
todolist = pd.DataFrame()
todolist = pd.merge(dataset, palette, on=['DataSet', 'FieldID'])
todolist.head(5)

# Create a List of distinct formula to apply

In [0]:
formulas = pd.DataFrame()
formulas = pd.DataFrame(todolist, columns=['KeyWord','PythonFormula'])
formulas = formulas.drop_duplicates()
formulas.head(10)

# Prepare Results Table

In [0]:
global bigresults
littletodolist = pd.DataFrame()
littletodolistworking = pd.DataFrame()
bigresults = pd.DataFrame()


In [0]:
# this was an important test as if you do not have copy you get an error as it believes
# you are trying to change a view
#littletodolist = todolist[(todolist.KeyWord == 'PickList')].copy()
#littletodolist.head(100)
#filterdf = pd.DataFrame()
#filterdf = todolist[(todolist.KeyWord == 'PickList')].copy()
#filterdf.head()

In [0]:
# littletodolist.loc[littletodolist.KeyWord == 'Text', 'Result'] = np.where(littletodolist['CellType'] == 'Text', 1, 0)

# Loop through each Formula and apply to the To Do List

In [0]:
todolist = todolist.rename(columns={'CellValue_x': 'CellValue'})
##todolist.columns

In [0]:
for index, row in formulas.iterrows():
  # print(row['KeyWord'],row['PythonFormula'])

  pf = row['PythonFormula']
  kw = row['KeyWord']
 
  try:
      pf1 = pf.replace("dataset", "littletodolist")
  except:
      pf1 = "1"
  littletodolist = todolist[(todolist.KeyWord == kw)].copy()
  formula2do = "littletodolist['Result'] = " + pf1
  print(kw)
  print(formula2do)
  exec(formula2do)
  ##print(littletodolist)
  ##try:
  ##bigresults = pd.concat([bigresults, littletodolist], ignore_index=True).drop_duplicates().reset_index(drop=True)
  bigresults = pd.concat([bigresults, littletodolist]) ##, ignore_index=True).drop_duplicates().reset_index(drop=True)
  ##except:
  #  bigresults = bigresults

In [0]:
littletodolist['Result'] = np.where(littletodolist['CellType'] == 'Text', 1, 0)

# Add a sequence to the Results for use in Reporting

In [0]:
bigresults['ResultID'] = bigresults.index

In [0]:
bigresults.head(5)

# Narrow down Results to useful columnns

In [0]:
usefulresults = pd.DataFrame()
usefulresults = pd.DataFrame(bigresults, columns=['DataSet','FieldID','RowID','ColumnID','CellValue','CellType','CheckPerformed', 'Result'])
usefulresults.head()

# Summarise Results

In [0]:
resultssummary = pd.DataFrame()
# index = pd.MultiIndex.from_arrays(arrays, names=['first', 'second'])
# resultssummary.head()

resultssummary = bigresults[['DataSet','FieldID','CheckPerformed', 'Result', 'ResultID']].groupby(['DataSet', 'FieldID', 'CheckPerformed']).agg(['count'])
resultssummary.head(5)

# Pivot Results

In [0]:
# rownamearray = pd.array(['DataSet','FieldID'], dtype=str)
# newct = pd.crosstab(bigresults.DataSet,bigresults.FieldID,bigresults.CheckPerformed,bigresults.Result,rownames=rownamearray, colnames=['Result'])

newct = bigresults.pivot_table(index = ['DataSet','FieldID','CheckPerformed'], values = 'ResultID',columns = 'Result', aggfunc = 'count')
newct.head()

# Plot Stack Bar

## Show Stack

In [0]:
newct.plot(kind='barh', stacked=True)